In [7]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import matplotlib.pyplot as plt

* from bs4 import BeautifulSoup me va a permitr hacer el scraping del html para ir elemento por elemento de lo que necesitamos encontrar
* import requests me permite hacer una solicitud de tipo get para que la pág me devuelva el cod html
* lxml etree me sirve para usar xpath dentro de beautifulsoup que me permite buscar y seleccionar (cuando la ruta obtenida es muy larga)teniendo en cuenta la estructura jerárquica del XML.
* pandas me permite exportar lo obtenido en un archivo con formato csv.

In [8]:
sitioweb = requests.get ("https://listado.mercadolibre.com.ar/aire-acondicionado")  

sitioweb.status_code

contenido = sitioweb.text

* sitioweb = coloco la web dond quiero extraer la data

* contenido = se convierte en texto lo obtenido de variable sitioweb

In [9]:
soup = BeautifulSoup (contenido, "lxml")

* esto me permite obtener el código html completo

In [11]:
articulo = soup.find_all("h2", attrs={"class":"ui-search-item__title shops__item-title"})
articulo = [i.text for i in articulo]

* articulo=soup.find_all("h2", attrs={"class":"ui-search-item__title shops__item-title"}) esto nos permite encontrar y extraer todos los títulos de los productos que hay en la clase que especificamos
* articulo =[i.text for i in titulos] nos permite justamente que si imprimimos los títulos, nos arroje como resultado cada uno de los productos en forma de texto.

In [12]:
dom = etree.HTML(str(soup))
precios = dom.xpath('//li[@class="ui-search-layout__item shops__layout-item"]//div[@class="ui-search-result__content-columns shops__content-columns"]//div[@class="ui-search-result__content-column ui-search-result__content-column--left shops__content-columns-left"]/div[1]/div//div[@class="ui-search-price__second-line shops__price-second-line"]//span[@class="price-tag-amount"]/span[2]')
precios = [i.text for i in precios]

* Esto me permite exraer y mostrar como texto finalmente con i.text cada uno de los precios de los productos.

In [14]:
urls = soup.find_all("a",attrs={"class": "ui-search-item__group__element shops__items-group-details ui-search-link"})
urls = [i.get("href") for i in urls]

In [15]:
siguiente = dom.xpath('//div [@class="ui-search-pagination shops__pagination-content"]/ul/li[contains(@class,"--next")]/a')[0].get("href")

* Esto nos permite encontrar el botón de siguiente para ir avanzando sobre las páginas. El [0].get ("href"), lo que me muestra si imprimo la variable siguiente, es justamente la url de la pág que se encuentra en la primer posición. 

In [16]:
ini = soup.find ("span", attrs={"class":'andes-pagination__link'}).text
ini=int(ini)

* Utilizo soup.find porque tengo uno solo atributo no hace falta utilizar xpath, auque podría usarlo que también funciona. Declaro .text para obetener el texto, como en en verdad este código me va imprimir el número de pag en la que me encuentro, lo que hago es declarar que la variable ini es un entero y si imprimo me arroja el número.

In [17]:
fin = soup.find("li", attrs={'class':"andes-pagination__page-count"})
fin = int(fin.text.split(" ")[1])

* En este caso se hace algo similar al códido de ini con el código que dice fin donde indicamos que estamos en la pág 40. Como lo que extraigo contiene como primer caracter la palabra de y yo sólo quiero obtener el n°40, declaro la variable como entero y le indico que tome y muestre si imprimo el 2 lugar por eso entre [] coloco el uno, para indicar la posición.

In [32]:
#Todo este código me va a permitir ir recorriendo cada una de las páginas e ir extrayendo lo que necesito en este caso los articulos y sus respectivos precios
#Creo listas donde se van a ir guardado todos los articulos de todas las pág a medida que se valla recorriendo el bucle y pasando de una pag a la otra 
lista_articulo = []
lista_precios = []
lista_url = []

In [34]:
siguiente = "https://listado.mercadolibre.com.ar/aire-acondicionado"

while True:
    r = requests.get(siguiente)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, "html.parser")
        #Titulos
        articulo = soup.find_all("h2", attrs={"class":"ui-search-item__title shops__item-title"})
        articulo = [i.text for i in articulo]
        lista_articulo.extend(articulo)
        #Precios
        dom = etree.HTML(str(soup))
        precios = dom.xpath('//li[@class="ui-search-layout__item shops__layout-item"]//div[@class="ui-search-result__content-columns shops__content-columns"]//div[@class="ui-search-result__content-column ui-search-result__content-column--left shops__content-columns-left"]/div[1]/div//div[@class="ui-search-price__second-line shops__price-second-line"]//span[@class="price-tag-amount"]/span[2]')
        precios = [i.text for i in precios]
        lista_precios.extend(precios)
        ini = soup.find ("span", attrs={"class":'andes-pagination__link'}).text
        ini=int(ini)
        fin = soup.find("li", attrs={'class':"andes-pagination__page-count"})
        fin = int(fin.text.split(" ")[1])
        #Urls
        urls = soup.find_all("a",attrs={"class": "ui-search-item__group__element shops__items-group-details ui-search-link"})
        urls = [i.get("href") for i in urls]
        lista_url.extend(urls)

    else:
        break
    print(ini,fin)

    if ini==fin:
        break
    siguiente = dom.xpath('//div [@class="ui-search-pagination shops__pagination-content"]/ul/li[contains(@class,"--next")]/a')[0].get("href")

1 40
2 40
3 40
4 40
5 40
6 40
7 40
8 40
9 40
10 40
11 40
12 40
13 40
14 40
15 40
16 40
17 40
18 40
19 40
20 40
21 40
22 40
23 40
24 40
25 40
26 40
27 40
28 40
29 40
30 40
31 40
32 40
33 40
34 40
35 40
36 40
37 40
38 40
39 40
40 40


In [70]:
datos = pd.DataFrame({"Articulos":lista_articulo, "Precios":lista_precios, "Urls":lista_url})
datos = pd.read_csv ("aire-acondicionado.csv", header=0)
datos

,Unnamed: 0,Articulos,Precios,Urls
0,0,Aire acondicionado Likon mini split frío/cal...,99.999,https://www.mercadolibre.com.ar/aire-acondicio...
1,1,Aire Acondicionado Likon Split Frío/calor3500w...,99.999,https://articulo.mercadolibre.com.ar/MLA-82220...
2,2,Aire Acondicionado Split Inverter Frío/calor S...,184.999,https://articulo.mercadolibre.com.ar/MLA-11984...
3,3,Aire acondicionado Philco split frío/calor 2...,103.999,https://www.mercadolibre.com.ar/aire-acondicio...
4,4,Aire acondicionado Philco split frío/calor 2...,99.999,https://www.mercadolibre.com.ar/aire-acondicio...
...,...,...,...,...
2009,2009,Aire Acondicionado Hyundai Inverter 3224w Frio...,152.999,https://articulo.mercadolibre.com.ar/MLA-11932...
2010,2010,Aire Acondicinado Samsung 2250 Fc Repuestos,3.000,https://articulo.mercadolibre.com.ar/MLA-89690...
2011,2011,Aire Surrey Piso Techo 3tr / 9000 Frigorías F/...,497.799,https://articulo.mercadolibre.com.ar/MLA-61816...
2012,2012,Aire Ac Multisplit Midea 21000fr Reparamos Arr...,800,https://articulo.mercadolibre.com.ar/MLA-11789...
